In [ ]:
import os
import progressbar
import networkx as nx
import numpy as np
import pandas as pd
import pickle
import biograph.graph_models as graph_models

os.chdir("../")

### Graph verification and preprocessing

In [ ]:
def load_graph(fn):
    with open(fn, "rb") as f:
        return pickle.load(f)
filenames = [fn for fn in os.listdir("graphs/contacts/") if ".pkl" in fn]
#filenames = ["3KEU.pkl"]
graphs = [load_graph("graphs/contacts/"+fn) for fn in filenames]
print("Loaded {} graphs".format(len(graphs)))

In [3]:
has_missing_data = []
for i, graph in enumerate(graphs):
    nodes_without_data = False
    for node_idx in graph.nodes:
        if "chain" not in graph.nodes[node_idx]:
            nodes_without_data = True
            break
    has_missing_data.append(nodes_without_data)

In [4]:
graphs = [graph for i, graph in enumerate(graphs) if not has_missing_data[i]]
len(graphs)

814

In [5]:
g = graphs[0]
for node_idx in g.nodes:
    if "x" not in g.nodes[node_idx] or g.nodes[node_idx]["x"] is None:
        print("Oops:", node_idx)

In [6]:
for i in progressbar.progressbar(range(len(graphs))):
    graphs[i] = graph_models.GraphModel.get_diffused_graph(graphs[i], steps=3)

100% (814 of 814) |######################| Elapsed Time: 0:20:57 Time:  0:20:57


In [ ]:
graphs[0].nodes["B:ILE:482"]

In [7]:
import biograph.constants
def touches_ligand(x):
    return x <= 4 or (x<=6 and np.random.binomial(1, 1-(x-4)/2) == 1)
dataframes = {} # chain_id to dataframe
for graph in progressbar.progressbar(graphs):
    df = graph_models.GraphModel.graph_to_dataframe(graph)
    # One hot for AAs
    for code3 in biograph.constants.AMINOACIDS_3 + ["UNK"]:
        df[code3] = (df.resname == code3).astype(np.int)
    
    #for i, coord in enumerate(["x", "y", "z"]):
    #    df[coord] = df.coord.map(lambda x: x[i])
        
    df["target"] = df.distance.map(touches_ligand).astype(np.int)
    df = df[[c for c in df.columns if "distance_" not in c]]
    df = df.drop(["resname", "distance"], axis=1)
    for chain in df.chain.unique():
        dataframes[chain] = df.loc[df.chain==chain].drop(["chain"], axis=1)

100% (814 of 814) |######################| Elapsed Time: 0:00:50 Time:  0:00:50


In [8]:
print([df.columns for df in list(dataframes.values())[0:1]])

[Index(['x_1', 'x_2', 'x_3', 'y_1', 'y_2', 'y_3', 'z_1', 'z_2', 'z_3',
       'bfactor_1', 'bfactor_2', 'bfactor_3', 'occupancy_1', 'occupancy_2',
       'occupancy_3', 'x', 'y', 'z', 'bfactor', 'occupancy', 'ALA', 'ARG',
       'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS',
       'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL', 'UNK',
       'target'],
      dtype='object')]


In [ ]:
# todo: class balance

## Example model

### Data splits and preprocessing

In [9]:
with open("names_groups.pkl", "rb")  as f:
    names, groups = pickle.load(f)

In [10]:
# sanity check: each protein's chains belong only to one cluster..?
counter = 0
for pdbid, chains in names.items():
    clusters = [groups[chain] for chain in chains if groups[chain] is not None]
    if len(set(clusters)) > 1:
        counter += 1
        print("There is a protein, {}, with different clusters different chains".format(pdbid))
        print("Groups not ignoring None:",
             [groups[chain] for chain in chains])
        continue
    

There is a protein, 2RD5, with different clusters different chains
Groups not ignoring None: [496, 496, 876, 876]
There is a protein, 5ECP, with different clusters different chains
Groups not ignoring None: [129, 720, 720, 129, 720, 720]
There is a protein, 6REP, with different clusters different chains
Groups not ignoring None: [973, 105, 271, 483, 556, 903, 834, 776, 963, 950, 896, 896, 896, 896, 896, 896, 896, 896, 896, 896, 480, 712, 989, 759, 498, 137, 137, 137, 130, 130, 130]
There is a protein, 2YCH, with different clusters different chains
Groups not ignoring None: [376, 1034]
There is a protein, 4WB5, with different clusters different chains
Groups not ignoring None: [325, 1030]
There is a protein, 6GJS, with different clusters different chains
Groups not ignoring None: [12, 837, 855]
There is a protein, 4EAK, with different clusters different chains
Groups not ignoring None: [939, 965, 471]
There is a protein, 2A40, with different clusters different chains
Groups not ignoring

In [ ]:
groups.get("3RRF_A") is not None

In [11]:
# Concatenate dataframes and assign groups
for chain in dataframes.keys():
    # usage of .get is intentional (fails on 5A99 that has "no sequences")
    dataframes[chain]["group"] = groups.get(chain) if groups.get(chain) is not None else np.nan

dataset = pd.concat(dataframes, ignore_index=True)
dataset = dataset.loc[dataset.group.notna()]
row_groups = dataset.group
row_target = dataset.target

dataset = dataset.drop(["target", "group"], axis=1)
dataset.head()

/home/joaquintz/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,ALA,ARG,ASN,ASP,CYS,GLN,GLU,GLY,HIS,ILE,...,x_2,x_3,y,y_1,y_2,y_3,z,z_1,z_2,z_3
0,0,0,0,0,1,0,0,0,0,0,...,26.6283,26.3555,24.246000,22.736750,24.9369,27.3292,85.348000,83.368500,82.8406,80.8652
1,0,0,0,0,0,0,1,0,0,0,...,28.0647,30.6745,20.773001,20.766333,22.2554,26.0605,82.264999,81.527167,78.684,77.1125
2,0,0,0,0,0,1,0,0,0,0,...,33.8856,34.5048,21.856001,22.676000,22.8771,24.5166,67.103996,66.899286,71.1405,74.2383
3,0,0,0,0,0,0,0,0,0,0,...,31.2572,34.0572,21.382000,22.894334,24.9892,23.5525,62.618999,66.596664,67.8135,72.9585
4,0,1,0,0,0,0,0,0,0,0,...,27.9988,27.3126,34.183998,31.594600,31.3326,30.4195,86.623001,88.286600,86.139,82.0163


In [12]:
#dataset = dataset.drop(["resname_2", "resname_3", "chain_2", "chain_3"], axis=1)
for objcol in dataset.select_dtypes("object").columns.tolist():
    dataset[objcol] = pd.to_numeric(dataset[objcol], errors="coerce")

In [13]:
# class %
row_target.sum()/row_target.shape[0]*100

1.3960014143687893

In [14]:
import sklearn.model_selection
import sklearn.metrics
import xgboost as xgb

param = {
    'eta': 0.3268179417171394,
    'max_depth': 8,
    'min_child_weight': 0.5460906834882965,
    'objective': "binary:logistic",
    'nthread': 4
}
groupk= sklearn.model_selection.GroupKFold(n_splits=5)
clf = xgb.XGBClassifier(**param)
res = sklearn.model_selection.cross_validate(
    clf, dataset, y=row_target, groups=row_groups,
    cv=groupk, n_jobs=3, scoring="roc_auc", verbose=1, return_estimator=True)
res

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:  7.6min finished


{'fit_time': array([262.54166961, 269.31639576, 266.70230889, 188.45811558,
        188.79723334]),
 'score_time': array([0.67999792, 0.59584165, 0.69067121, 0.52409148, 0.45296025]),
 'estimator': (XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bynode=1, colsample_bytree=1, eta=0.3268179417171394,
                gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=8,
                min_child_weight=0.5460906834882965, missing=nan,
                n_estimators=100, n_jobs=1, nthread=4,
                objective='binary:logistic', random_state=0, reg_alpha=0,
                reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
                subsample=1, verbosity=1),
  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bynode=1, colsample_bytree=1, eta=0.3268179417171394,
                gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=8,
                min_child_weight=0.5460

In [20]:
from scipy.stats import uniform
import time
import dask_ml.model_selection as dcl
param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
param['nthread'] = 3
param['eval_metric'] = 'auc'

groupk= sklearn.model_selection.GroupKFold(n_splits=5)
clf = xgb.XGBClassifier(**param)

#sklearn.model_selection.cross_validate(
#    clf, dataset, y=row_target, groups=row_groups,
#    cv=groupk, n_jobs=1, scoring="roc_auc", verbose=1)

#clf = sklearn.model_selection.RandomizedSearchCV(
clf = dcl.RandomizedSearchCV(
    clf, {"max_depth": list(range(3,10)), 
          "min_child_weight": uniform(loc=0.5, scale=1.5), # ~U(loc, loc+scale)
          "eta": uniform(loc=0.2, scale=0.3)},
    cv=groupk, n_iter=50, scoring="roc_auc", n_jobs=3) #verbose=5

start = time.time()
search = clf.fit(dataset, row_target, row_groups)
end = time.time()

In [21]:
end-start

17794.97421002388

In [22]:
search.cv_results_

{'params': [{'eta': 0.41227546594708686,
   'max_depth': 3,
   'min_child_weight': 1.4030132810619707},
  {'eta': 0.2576036746649024,
   'max_depth': 7,
   'min_child_weight': 0.6716751729577174},
  {'eta': 0.46148782318055925,
   'max_depth': 7,
   'min_child_weight': 1.9908909843298073},
  {'eta': 0.3474890599852592,
   'max_depth': 8,
   'min_child_weight': 1.001553505903734},
  {'eta': 0.47982197397374887,
   'max_depth': 9,
   'min_child_weight': 1.15672711494252},
  {'eta': 0.4988947574231741,
   'max_depth': 7,
   'min_child_weight': 0.969090273093909},
  {'eta': 0.4753658857935113,
   'max_depth': 5,
   'min_child_weight': 0.8030384241846243},
  {'eta': 0.2513284337657757,
   'max_depth': 4,
   'min_child_weight': 1.4028049769657565},
  {'eta': 0.21555863655624,
   'max_depth': 6,
   'min_child_weight': 0.5508744638880556},
  {'eta': 0.29894930672011616,
   'max_depth': 9,
   'min_child_weight': 1.5187639027525812},
  {'eta': 0.3761814559161182,
   'max_depth': 4,
   'min_child

In [23]:
search.best_params_, search.best_score_

({'eta': 0.21891302744649455,
  'max_depth': 9,
  'min_child_weight': 1.217264621371775},
 0.7415439252240505)

In [24]:
search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.21891302744649455,
              eval_metric='auc', gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=9, min_child_weight=1.217264621371775, missing=None,
              n_estimators=100, n_jobs=1, nthread=3,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

In [ ]:

xgb.plot_importance(search.best_estimator_, max_num_features=14)

In [26]:
estimator = search.best_estimator_
preds = estimator.predict(dataset)
print("Precision: ", sklearn.metrics.precision_score(row_target, preds))
print("Recall: ", sklearn.metrics.recall_score(row_target, preds))
pred_class = np.copy(preds)
pred_class[pred_class > 0.5] = 1
pred_class[pred_class <= 0.5] = 0
print("AUC: ", sklearn.metrics.roc_auc_score(row_target, preds))
print("Confusion matrix: \n", sklearn.metrics.confusion_matrix(row_target, preds))

Precision:  0.9504080351537979
Recall:  0.08775285457601577
AUC:  0.5438440139465125
Confusion matrix: 
 [[1218555      79]
 [  15739    1514]]
